In [0]:
%run "/Workspace/Shared/tenon/connectors"

In [0]:
%run "/Workspace/Shared/tenon/datacleaning" 

In [0]:
adlsconnector
pati_df  = readsilverdata('Patient_S.csv')
hospi_df = readsilverdata('Hospital_S.csv')
claims_df = readsilverdata('Claims_S.csv')
subscrib_df = readsilverdata('Subscriber_S.csv')
subgroup_df = readsilverdata('SubGroup_S.csv')
groups_df = readsilverdata('Group_S.csv')
dises_df = readsilverdata('Disease_S.csv')
hospi_df = hospi_df.withColumnRenamed('city','hospital_city') #renaming the city column to match the hospital_city column
pati_df = pati_df.withColumnRenamed('city','patient_city') #renaming the city column to match the patient_city column
pati_hospi_df = pati_df.join(hospi_df, pati_df.hospital_id == hospi_df.Hospital_id, 'inner') #joining the two dataframes
pati_hospi_df = pati_hospi_df.withColumnRenamed('disease_name','disease_name_new')#renaming disese columns
pati_hospi_df = pati_hospi_df.drop('Hospital_id') #reomoving the hospital_id column
claims_df = claims_df.drop('patient_id') #removing extra patient_id column
pati_hospi_claim_df= pati_hospi_df.join(claims_df, pati_hospi_df.Patient_id == claims_df.patinet_id, 'inner') #joining the claims data with the patient-hospital data 
pati_hospi_claim_subscriber_df= pati_hospi_claim_df.join(subscrib_df, pati_hospi_claim_df['sub_id'] == subscrib_df['sub_id'], 'inner') #join the subscriber data with the claims-patient-hospital data
pati_hospi_claim_subscriber_df = pati_hospi_claim_subscriber_df.drop('patient_id','sub_id', 'first_name', 'last_name',  'Gender', 'Country', 'City', ) #dropping duplicate columns
subgroup_df = subgroup_df.withColumnRenamed('subgrp_id','grp_id')#renaming the column subgrp_id to grp_id
pati_hospi_claim_subscriber_subgrp_df= pati_hospi_claim_subscriber_df.join(subgroup_df, pati_hospi_claim_subscriber_df['Subgrp_id'] == subgroup_df['subgrp_sk'], 'inner') # join the subgroup data with the claims-patient-hospital-subscriber data
pati_hospi_claim_subscriber_subgrp_df = pati_hospi_claim_subscriber_subgrp_df.drop('disease_name') #dropping disesname
pati_hospi_claim_subscriber_subgrp_disese_df = pati_hospi_claim_subscriber_subgrp_df.join(dises_df, pati_hospi_claim_subscriber_subgrp_df['subgrp_sk'] == dises_df['subgrp_id'], 'inner')  #join the disease data with the claims-patient-hospital-subscriber-subgroup data
pati_hospi_claim_subscriber_subgrp_disese_df = pati_hospi_claim_subscriber_subgrp_disese_df.drop('subgrp_id')
pati_hospi_claim_subscriber_subgrp_disese_group_df = pati_hospi_claim_subscriber_subgrp_disese_df.join(groups_df, pati_hospi_claim_subscriber_subgrp_disese_df['grp_id'] == groups_df['grp_id'], 'inner') #join the group data with the claims-patient-hospital-subscriber-subgroup-disease data
pati_hospi_claim_subscriber_subgrp_disese_group_df = pati_hospi_claim_subscriber_subgrp_disese_group_df.drop('grp_id','city','country','Zip Code','Street','age') #reomving the duplicate columns


In [0]:
write2gold(pati_hospi_claim_subscriber_subgrp_disese_group_df, 'tenon_final_data.csv')  #writing to final gold layer

CSV file wrtitten to abfss://tenon@tenonadls.dfs.core.windows.net/gold//tenon_final_data.csv


'abfss://tenon@tenonadls.dfs.core.windows.net/gold//tenon_final_data.csv'

In [0]:
pati_hospi_claim_subscriber_subgrp_disese_group_df.write.format("delta").option("mergeSchema", "true").saveAsTable("tenon_final_data1")